<a href="https://colab.research.google.com/github/calebsiyasiya/MIT-805-Project/blob/main/MIT_805_Assignment_Part_2_MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pre-Processing**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Mount Google drive where data is stored
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
csv_data = pd.read_csv('/content/drive/MyDrive/MIT 805 Semester Project/store_zara.csv')

csv_data.head(3)

,brand,url,sku,name,description,price,currency,images,scraped_at,terms,section,error,image_downloads
0,Zara,https://www.zara.com/us/en/basic-puffer-jacket...,272145190-250-2,BASIC PUFFER JACKET,Puffer jacket made of tear-resistant ripstop f...,19.99,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:05.654618,jackets,MAN,NaN,"['e8e4ae57-8b72-44ff-aa5d-84de3ed37d9e', '0cb3..."
1,Zara,https://www.zara.com/us/en/tuxedo-jacket-p0889...,324052738-800-46,TUXEDO JACKET,Straight fit blazer. Pointed lapel collar and ...,169.00,USD,['https://static.zara.net/photos///2024/V/0/1/...,2024-02-19T08:50:06.590930,jackets,MAN,NaN,"['b42b0725-cfe2-4748-af3e-3abe590d83cd', 'ed9c..."
2,Zara,https://www.zara.com/us/en/slim-fit-suit-jacke...,335342680-800-44,SLIM FIT SUIT JACKET,Slim fit jacket. Notched lapel collar. Long sl...,129.00,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:07.301419,jackets,MAN,NaN,"['c27bdddf-2f9c-4693-976f-0d1272212c12', '07e6..."


In [21]:
#Remove columns that won't be used in analysis
csv_data = csv_data.drop(columns=['brand','url','scraped_at','error','images'],axis=1)

In [22]:
#Check info about data set
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sku              3129 non-null   object 
 1   name             3065 non-null   object 
 2   description      3059 non-null   object 
 3   price            3129 non-null   float64
 4   currency         3129 non-null   object 
 5   terms            3129 non-null   object 
 6   section          3129 non-null   object 
 7   image_downloads  3129 non-null   object 
dtypes: float64(1), object(7)
memory usage: 195.7+ KB


In [23]:
#Some columns have ull values. Check rows with null values
csv_data[csv_data['name'].isnull()]

,sku,name,description,price,currency,terms,section,image_downloads
81,336378923-700-2,NaN,NaN,129.0,USD,jackets,MAN,"['7783d9a4-9913-4ac1-acb3-7b8d64ef1ac7', 'dc7e..."
146,330734744-800-2,NaN,NaN,129.0,USD,jackets,MAN,"['85e083c5-226c-4b18-a64a-39f931db9a71', 'f786..."
256,329893545-800-2,NaN,NaN,69.9,USD,pants,MAN,"['ca24a94e-c14b-4584-b5d3-a3b2295384ae', '0258..."
272,335677693-800-2,NaN,NaN,49.9,USD,pants,MAN,"['a75c7489-327a-408b-a904-9ba4d5423b15', 'dd0c..."
273,326561477-826-2,NaN,NaN,49.9,USD,pants,MAN,"['0824933b-fbf9-4890-ada4-6f4c08325978', '690a..."
...,...,...,...,...,...,...,...,...
2657,320060926-807-35,NaN,NaN,49.9,USD,shoes,WOMAN,"['978ad17f-b8f5-4302-bea2-521657049ae4', 'e649..."
3028,331694122-800-1,NaN,NaN,45.9,USD,coats,WOMAN,"['eb07c4e8-d938-4282-a4de-f938c7ff94fe', 'aa14..."
3116,342922122-999-99,NaN,NaN,35.9,USD,bags,WOMAN,"['2d530985-5832-4bd9-b586-8bd300296807', 'bd9b..."
3117,336468928-303-3,NaN,NaN,17.9,USD,bags,WOMAN,"['a77bc4a8-0c31-4dfe-9fb1-42d1fede61ff', '4099..."


In [24]:
#Check unique values in 'terms' columns
csv_data['terms'].unique()

array(['jackets', 'puffers', 'pants', 'jeans', 'sweaters', 'cardigans',
       'hoodies', 'sweatshirts', 't-shirts', 'overshirts', 'linen',
       'shorts', 'suits', 'blazers', 'tracksuits', 'coats', 'shoes',
       'bags', 'dresses', 'skirts', 'tops', 'bodysuits', 'knitwear'],
      dtype=object)

In [25]:
#Collect indices of rows with no name or description
blank_indices = csv_data[csv_data['name'].isnull()].index
blank_indices

Index([  81,  146,  256,  272,  273,  279,  513,  582,  707,  833, 1132, 1139,
       1159, 1180, 1202, 1267, 1272, 1275, 1295, 1299, 1305, 1312, 1324, 1340,
       1342, 1388, 1464, 1467, 1494, 1511, 1517, 1526, 1620, 1652, 1712, 1714,
       1719, 1726, 1812, 1815, 1818, 1821, 1822, 1845, 1846, 1857, 1862, 2038,
       2044, 2048, 2052, 2059, 2060, 2351, 2367, 2460, 2475, 2496, 2652, 2657,
       3028, 3116, 3117, 3127],
      dtype='int64')

In [26]:
#Fill blan name dsecriptions with terms
for idx in blank_indices:
  csv_data.loc[idx,'name'] = csv_data.loc[idx,'terms']
  csv_data.loc[idx,'description'] = csv_data.loc[idx,'terms']

In [27]:
blank_indices = list(blank_indices)

csv_data.iloc[blank_indices]

,sku,name,description,price,currency,terms,section,image_downloads
81,336378923-700-2,jackets,jackets,129.0,USD,jackets,MAN,"['7783d9a4-9913-4ac1-acb3-7b8d64ef1ac7', 'dc7e..."
146,330734744-800-2,jackets,jackets,129.0,USD,jackets,MAN,"['85e083c5-226c-4b18-a64a-39f931db9a71', 'f786..."
256,329893545-800-2,pants,pants,69.9,USD,pants,MAN,"['ca24a94e-c14b-4584-b5d3-a3b2295384ae', '0258..."
272,335677693-800-2,pants,pants,49.9,USD,pants,MAN,"['a75c7489-327a-408b-a904-9ba4d5423b15', 'dd0c..."
273,326561477-826-2,pants,pants,49.9,USD,pants,MAN,"['0824933b-fbf9-4890-ada4-6f4c08325978', '690a..."
...,...,...,...,...,...,...,...,...
2657,320060926-807-35,shoes,shoes,49.9,USD,shoes,WOMAN,"['978ad17f-b8f5-4302-bea2-521657049ae4', 'e649..."
3028,331694122-800-1,coats,coats,45.9,USD,coats,WOMAN,"['eb07c4e8-d938-4282-a4de-f938c7ff94fe', 'aa14..."
3116,342922122-999-99,bags,bags,35.9,USD,bags,WOMAN,"['2d530985-5832-4bd9-b586-8bd300296807', 'bd9b..."
3117,336468928-303-3,bags,bags,17.9,USD,bags,WOMAN,"['a77bc4a8-0c31-4dfe-9fb1-42d1fede61ff', '4099..."


In [28]:
#Check null values again
csv_data.isnull().sum()

,0
sku,0
name,0
description,6
price,0
currency,0
terms,0
section,0
image_downloads,0


In [ ]:
#Save modified data frame
csv_data.to_csv('/content/drive/MyDrive/MIT 805 Semester Project/store_zara_modified.csv',index=False)

# **Processing**

In this section of the notebook, two computations are performed using PySpark.The first computation reads in the modified store_zara_modified.csv file and expands the data frame by spliting the images stored as lists in the 'image_downloads' column into seperate rows. This was required in order to create the visualization shown later.

The second PySpark computation is used to process the 25000 images of the various fashion items. The images are read into an RDD before performing a transformation on the data to determine the brightness of each image. The resulting output is then saved to a data frame which in turn is saved as a CSV file. This CSV file will be combined with the CSV file obtained in the pre-processing to create the visualizations of interest

**Set-up Spark Session**

In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=a5a82f9826d7f781b5a85f98d12ae1d48b5c72089986206c3e9f169cb04770d8
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from PIL import Image
import os
import sys

In [ ]:
#Create spark session
spark = SparkSession.builder \
    .appName("mySparkApp") \
    .config("spark.driver.memory", "10g")\
    .getOrCreate()

In [ ]:
# Get driver memory configuration
driver_memory = spark.conf.get("spark.driver.memory")
print(f"Driver Memory: {driver_memory}")

Driver Memory: 10g


In [ ]:
#Configure environment variable path
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

**First Computation: CSV file Processing**

In [ ]:
#Read in modified csv file from pre-processing
csv_path = '/content/drive/MyDrive/MIT 805 Semester Project/store_zara_modified.csv'
df1 = spark.read.csv(csv_path, header=True, inferSchema=True)

In [ ]:
df1.show(5)

+----------------+--------------------+--------------------+-----+--------+-------+-------+--------------------+
|             sku|                name|         description|price|currency|  terms|section|     image_downloads|
+----------------+--------------------+--------------------+-----+--------+-------+-------+--------------------+
| 272145190-250-2| BASIC PUFFER JACKET|Puffer jacket mad...|19.99|     USD|jackets|    MAN|['e8e4ae57-8b72-4...|
|324052738-800-46|       TUXEDO JACKET|Straight fit blaz...|169.0|     USD|jackets|    MAN|['b42b0725-cfe2-4...|
|335342680-800-44|SLIM FIT SUIT JACKET|Slim fit jacket. ...|129.0|     USD|jackets|    MAN|['c27bdddf-2f9c-4...|
|328303236-420-44| STRETCH SUIT JACKET|Slim fit jacket m...|129.0|     USD|jackets|    MAN|['0cbc84c8-f9ff-4...|
| 312368260-800-2| DOUBLE FACED JACKET|Jacket made of fa...|139.0|     USD|jackets|    MAN|['1ccc9c87-49c9-4...|
+----------------+--------------------+--------------------+-----+--------+-------+-------+-----

In [ ]:
#Check schema for df1
df1.printSchema()

root
 |-- sku: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- terms: string (nullable = true)
 |-- section: string (nullable = true)
 |-- image_downloads: string (nullable = true)



In [ ]:
#Convert price and image_downloads columns to correct format
df1 = df1.withColumn("price", df1["price"].cast("float"))\
      .withColumn("image_downloads", split(df1["image_downloads"], ","))

In [ ]:
df1.show(5)

+----------------+--------------------+--------------------+-----+--------+-------+-------+--------------------+
|             sku|                name|         description|price|currency|  terms|section|     image_downloads|
+----------------+--------------------+--------------------+-----+--------+-------+-------+--------------------+
| 272145190-250-2| BASIC PUFFER JACKET|Puffer jacket mad...|19.99|     USD|jackets|    MAN|[['e8e4ae57-8b72-...|
|324052738-800-46|       TUXEDO JACKET|Straight fit blaz...|169.0|     USD|jackets|    MAN|[['b42b0725-cfe2-...|
|335342680-800-44|SLIM FIT SUIT JACKET|Slim fit jacket. ...|129.0|     USD|jackets|    MAN|[['c27bdddf-2f9c-...|
|328303236-420-44| STRETCH SUIT JACKET|Slim fit jacket m...|129.0|     USD|jackets|    MAN|[['0cbc84c8-f9ff-...|
| 312368260-800-2| DOUBLE FACED JACKET|Jacket made of fa...|139.0|     USD|jackets|    MAN|[['1ccc9c87-49c9-...|
+----------------+--------------------+--------------------+-----+--------+-------+-------+-----

In [ ]:
#Expand each list in the image_downloads column to store each image name as a string in a row of its own
df1 = df1.withColumn("image_downloads", explode(col("image_downloads")))
#Remove square brackets from image names
df1 = df1.withColumn("image_downloads", regexp_replace(col("image_downloads"), "\\[", ""))
df1 = df1.withColumn("image_downloads", regexp_replace(col("image_downloads"), "\\]", ""))

In [ ]:
df1.show(5)

+---------------+-------------------+--------------------+-----+--------+-------+-------+--------------------+
|            sku|               name|         description|price|currency|  terms|section|     image_downloads|
+---------------+-------------------+--------------------+-----+--------+-------+-------+--------------------+
|272145190-250-2|BASIC PUFFER JACKET|Puffer jacket mad...|19.99|     USD|jackets|    MAN|'e8e4ae57-8b72-44...|
|272145190-250-2|BASIC PUFFER JACKET|Puffer jacket mad...|19.99|     USD|jackets|    MAN| '0cb3791f-ebc3-4...|
|272145190-250-2|BASIC PUFFER JACKET|Puffer jacket mad...|19.99|     USD|jackets|    MAN| '6831411d-91a5-4...|
|272145190-250-2|BASIC PUFFER JACKET|Puffer jacket mad...|19.99|     USD|jackets|    MAN| '184c9f8e-4765-4...|
|272145190-250-2|BASIC PUFFER JACKET|Puffer jacket mad...|19.99|     USD|jackets|    MAN| '037c90a2-abef-4...|
+---------------+-------------------+--------------------+-----+--------+-------+-------+--------------------+
o

In [ ]:
#save df1 as csv file
df1.write.csv("/content/drive/MyDrive/MIT 805 Semester Project/store_zara_modified_expanded.csv", header=True)

In [ ]:
# Stop Spark session
spark.stop()

**Second Computation: Image Processing**

In order to process the images, two custom functions are defined:

1.   calculate_brightenss: This function calculates the brightness of each image by converting the image to gray scale and calculating a bit-scale value for the brightness of the image
2.   process_image: This function reads in each image,calculates its brightness using the calculate_brightness function and returns this as a row with the image name in the first column and brightness in the second column



In [6]:
#Create spark session
spark = SparkSession.builder \
    .appName("mySparkApp") \
    .config("spark.driver.memory", "10g")\
    .getOrCreate()

In [7]:
#Configure ebvironment variable path
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [8]:
#Function to calculate brightness of an image
def calculate_brightness(image):
    image_array = np.array(image)
    pixel_values = image_array.flatten()
    # Convert list to NumPy array for calculation
    total_brightness = np.sum(pixel_values)
    num_pixels = len(pixel_values)
    #Prevent division by 0
    if num_pixels == 0:
        return 0.0
    else:
        return total_brightness / num_pixels

#Function to process image and return its brightness
def process_image(image_path):
    with Image.open(image_path) as img:
      brightness = calculate_brightness(img)
      return (image_path, brightness)



In [9]:
# Create path to images folder
images_folder = "/content/drive/MyDrive/MIT 805 Semester Project/zara_images"

In [10]:
# Store the names of each image file in a list
image_files = [os.path.join(images_folder, f) for f in os.listdir(images_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [11]:
#Check result
image_files[0:5]

['/content/drive/MyDrive/MIT 805 Semester Project/zara_images/f5b4a03a-7bcc-4482-9f61-9a845121c778.jpg',
 '/content/drive/MyDrive/MIT 805 Semester Project/zara_images/f5bd6c38-0577-4a4c-ad1c-d5ad55a9fc0d.jpg',
 '/content/drive/MyDrive/MIT 805 Semester Project/zara_images/f5b7ca89-7c4d-49d0-941a-1ce7b2c55a73.jpg',
 '/content/drive/MyDrive/MIT 805 Semester Project/zara_images/f587e1b4-6176-4f7a-b8b0-79ce80a90d16.jpg',
 '/content/drive/MyDrive/MIT 805 Semester Project/zara_images/f5a9a810-11f5-40b1-9a06-8e164387d91e.jpg']

In [12]:
len(image_files)

25134

In [13]:
# Create RDD from list of images and set number of partitions to 10
num_partitions = 10  # Adjust based on your environment
rdd = spark.sparkContext.parallelize(image_files, num_partitions)
#Create RDD transformation for obtaining brightness of each image using the process_image function defined above
results = rdd.map(process_image)

In [14]:
schema = StructType([
    StructField("filename", StringType(), True),
    StructField("brightness", FloatType(), True)  # Define brightness as FloatType
])

In [16]:
# Trigger spark action to execute transformations: Convert RDD to DataFrame
df2 = spark.createDataFrame(results.map(lambda row: (row[0], float(row[1]))), schema=schema)
#df2 = df2.withColumn("brightness", col("brightness").cast(FloatType()))
df2.show(5)

+--------------------+----------+
|            filename|brightness|
+--------------------+----------+
|/content/drive/My...| 168.65302|
|/content/drive/My...|  191.0412|
|/content/drive/My...|  8.155149|
|/content/drive/My...| 191.99742|
|/content/drive/My...|  85.51321|
+--------------------+----------+
only showing top 5 rows



In [19]:
#Save resulting data frame as a csv file
df2.write.csv("/content/drive/MyDrive/MIT 805 Semester Project/image_processing_output", header=True)